In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

2022-12-22 23:08:00.847386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 23:08:01.853794: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-22 23:08:01.853848: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-22 23:08:01.853853: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Имеем полный текст книги в файл ```wonderland.txt``` с обрезанными колонтитулами

Считываем файл, приводим все символы к нижнему регистру и приводим их к числовым значениям:

In [3]:
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

Просуммируем набор данных:

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  144572
Total Vocab:  49


Разделим весь текст на последовательности длиной в 100 символов:

In [5]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  144472


Преобразуем входную последовательность к форме [образцы, временные шаги, особенности] и преобразуем символы в одну кодировку:

In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Определяем LSTM сеть:

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

2022-12-22 23:09:02.698779: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-22 23:09:02.698813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (keesaev-asus-ubuntu): /proc/driver/nvidia/version does not exist
2022-12-22 23:09:02.699218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Создадим колбэк Checkpoint для сохранения весов на каждой эпохе:

In [8]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Обучение модели:

In [12]:
import tensorflow

with tensorflow.device("/GPU:0"):
    model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1129/1129 [==============================] - ETA: 0s - loss: 2.9920
Epoch 1: loss improved from inf to 2.99203, saving model to weights-improvement-01-2.9920.hdf5
1129/1129 [==============================] - 98s 87ms/step - loss: 2.9920
Epoch 2/20
1129/1129 [==============================] - ETA: 0s - loss: 2.7983
Epoch 2: loss improved from 2.99203 to 2.79834, saving model to weights-improvement-02-2.7983.hdf5
1129/1129 [==============================] - 105s 93ms/step - loss: 2.7983
Epoch 3/20
1129/1129 [==============================] - ETA: 0s - loss: 2.7052
Epoch 3: loss improved from 2.79834 to 2.70520, saving model to weights-improvement-03-2.7052.hdf5
1129/1129 [==============================] - 100s 89ms/step - loss: 2.7052
Epoch 4/20
1129/1129 [==============================] - ETA: 0s - loss: 2.6342
Epoch 4: loss improved from 2.70520 to 2.63421, saving model to weights-improvement-04-2.6342.hdf5
1129/1129 [==============================] - 102s 90ms/step - loss: 

Словарь для преобразования чисел обратно в символы:

In [13]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

Генерация текста:

In [14]:
from sys import stdout

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" 
alice said nothing; she had sat down with her face in her hands,
wondering if anything would _ever_ "
 toee i soon the while rabbit  she hor oo the whrle the was oo the toeer, and she was tooting to the tooe, and the woiee war soi time tith the world beain, and saed to the horphon. and alice was sor oo the woile oo the toede oo the was of the sabbet  she had not the hir  and the qooe turted her haad io a lote toiee 
the was a little oo too that she was soiee to hiree the was soiee to heree the was soie th the toeer, and she was aoling to the kotk of the courd  and saed to the horphon. and alice was sor oo the woile oo the toede oo the was of the sabbet  she had not the hir  and the qooe turted her haad io a lote toiee 
the was a little oo too that she was soiee to hiree the was soiee to heree the was soie th the toeer, and she was aoling to the kotk of the courd  and saed to the horphon. and alice was sor oo the woile oo the toede oo the was of the sabbet  she had not the hir 

Введём пользовательский ```callback```, в котором определим метод ```on_epoch_end```, в котором в канце каждой эпохи будем загружать чекпоинт, собирать из него модель, и генерировать 100 символов текста:

In [54]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("GENERATED TEXT (100 symbols):")

        tmp_model = Sequential([
            LSTM(256, input_shape=(X.shape[1], X.shape[2])),
            Dropout(0.2),
            Dense(y.shape[1], activation='softmax')
        ])

        filepath = "last_weights.hdf5"
        tmp_model.load_weights(filepath)
        tmp_model.compile(loss='categorical_crossentropy', optimizer='adam')
        
        # pick a random seed
        start = numpy.random.randint(0, len(dataX)-1)
        pattern = dataX[start]
        # generate characters
        for i in range(100):
            x = numpy.reshape(pattern, (1, len(pattern), 1))
            x = x / float(n_vocab)
            prediction = tmp_model.predict(x, verbose=0)
            index = numpy.argmax(prediction)
            result = int_to_char[index]
            seq_in = [int_to_char[value] for value in pattern]
            sys.stdout.write(result)
            pattern.append(index)
            pattern = pattern[1:len(pattern)]
        print("\nDone.")

In [57]:
from tensorflow.keras.callbacks import TensorBoard

filepath = "last_weights.hdf5"

model2 = Sequential([
    LSTM(256, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    Dense(y.shape[1], activation='softmax')
])
model2.compile(loss='categorical_crossentropy', optimizer='adam')

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

tensorboard_callback = TensorBoard(histogram_freq=1)

hist = model2.fit(X, y, epochs=10, batch_size=128, callbacks=[checkpoint, CustomCallback(), tensorboard_callback])

Epoch 1/10
1129/1129 [==============================] - ETA: 0s - loss: 3.0288
Epoch 1: loss improved from inf to 3.02881, saving model to last_weights.hdf5
GENERATED TEXT (100 symbols):
  ao th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th
Done.
1129/1129 [==============================] - 114s 100ms/step - loss: 3.0288
Epoch 2/10
1129/1129 [==============================] - ETA: 0s - loss: 2.8299
Epoch 2: loss improved from 3.02881 to 2.82990, saving model to last_weights.hdf5
GENERATED TEXT (100 symbols):
 toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe
Done.
1129/1129 [==============================] - 116s 102ms/step - loss: 2.8299
Epoch 3/10
1129/1129 [==============================] - ETA: 0s - loss: 2.7254
Epoch 3: loss improved from 2.82990 to 2.72543, saving model to last_weights.hdf5
GENERATED TEXT (100 symbols):
 toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe toe 

In [64]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
